In [28]:
import torch
import torch.nn as nn  #神经网络工具箱torch.nn 
import torch.nn.functional as F  #神经网络函数torch.nn.functional
import torch.utils.data as tud  #Pytorch读取训练集需要用到torch.utils.data类
from torch.nn.parameter import Parameter  #参数更新和优化函数

from collections import Counter #Counter 计数器
import numpy as np 
import random
import math 

import pandas as pd
import scipy #SciPy是基于NumPy开发的高级模块，它提供了许多数学算法和函数的实现
import sklearn
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度函数

In [29]:
USE_CUDA = torch.cuda.is_available() #有GPU可以用

# 为了保证实验结果可以复现，我们经常会把各种random seed固定在某一个值
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)

In [30]:
# 设定一些超参数   
K = 100 # number of negative samples 负样本随机采样数量
C = 3 # nearby words threshold 指定周围三个单词进行预测
NUM_EPOCHS = 2 # The number of epochs of training 迭代轮数
MAX_VOCAB_SIZE = 30000 # the vocabulary size 词汇表多大
BATCH_SIZE = 128 # the batch size 每轮迭代1个batch的数量
LEARNING_RATE = 0.2 # the initial learning rate #学习率
EMBEDDING_SIZE = 100 #词向量维度

In [31]:
LOG_FILE = "word-embedding.log"

# tokenize函数，把一篇文本转化成一个个单词
def word_tokenize(text): 
    return text.split()

- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [32]:
with open('./text8/text8/text8.train.txt','r',encoding='utf-8') as file:
    text = file.read()

In [34]:
#分词，在这里类似于text.split()
text = [w for w in word_tokenize(text.lower())]

In [35]:
# 字典格式，把（MAX_VOCAB_SIZE-1）个最频繁出现的单词取出来，-1是留给不常见的单词
vocab = dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))

In [36]:
print(len(vocab))

29999


In [37]:
# unk表示不常见单词数=总单词数-常见单词数
vocab['<unk>'] = len(text) - np.sum(list(vocab.values()))
# 这里计算得到的vocab['<unk>']=29999

In [40]:
#取出字典的所有单词key
idx_to_word = [word for word in vocab.keys()]

In [41]:
# idx_to_word

['the',
 'of',
 'and',
 'one',
 'in',
 'a',
 'to',
 'zero',
 'nine',
 'two',
 'is',
 'as',
 'eight',
 'for',
 's',
 'five',
 'three',
 'was',
 'by',
 'that',
 'four',
 'six',
 'seven',
 'with',
 'on',
 'are',
 'it',
 'from',
 'or',
 'his',
 'an',
 'be',
 'this',
 'he',
 'at',
 'which',
 'not',
 'also',
 'have',
 'were',
 'has',
 'but',
 'other',
 'their',
 'its',
 'first',
 'they',
 'had',
 'some',
 'more',
 'all',
 'can',
 'most',
 'been',
 'such',
 'who',
 'many',
 'new',
 'there',
 'used',
 'after',
 'american',
 'when',
 'time',
 'into',
 'these',
 'only',
 'see',
 'may',
 'than',
 'i',
 'world',
 'b',
 'd',
 'would',
 'no',
 'however',
 'between',
 'about',
 'over',
 'states',
 'years',
 'war',
 'people',
 'united',
 'during',
 'known',
 'if',
 'called',
 'use',
 'th',
 'often',
 'system',
 'so',
 'history',
 'state',
 'will',
 'up',
 'while',
 'where',
 'english',
 'city',
 'being',
 'then',
 'any',
 'under',
 'out',
 'both',
 'made',
 'e',
 'well',
 'them',
 'government',
 'numb

In [42]:
#取出所有单词的单词和对应的索引，索引值与单词出现次数相反，最常见单词索引为0
word_to_idx = {word:i for i,word in enumerate(idx_to_word)}

In [43]:
# word_to_idx

{'the': 0,
 'of': 1,
 'and': 2,
 'one': 3,
 'in': 4,
 'a': 5,
 'to': 6,
 'zero': 7,
 'nine': 8,
 'two': 9,
 'is': 10,
 'as': 11,
 'eight': 12,
 'for': 13,
 's': 14,
 'five': 15,
 'three': 16,
 'was': 17,
 'by': 18,
 'that': 19,
 'four': 20,
 'six': 21,
 'seven': 22,
 'with': 23,
 'on': 24,
 'are': 25,
 'it': 26,
 'from': 27,
 'or': 28,
 'his': 29,
 'an': 30,
 'be': 31,
 'this': 32,
 'he': 33,
 'at': 34,
 'which': 35,
 'not': 36,
 'also': 37,
 'have': 38,
 'were': 39,
 'has': 40,
 'but': 41,
 'other': 42,
 'their': 43,
 'its': 44,
 'first': 45,
 'they': 46,
 'had': 47,
 'some': 48,
 'more': 49,
 'all': 50,
 'can': 51,
 'most': 52,
 'been': 53,
 'such': 54,
 'who': 55,
 'many': 56,
 'new': 57,
 'there': 58,
 'used': 59,
 'after': 60,
 'american': 61,
 'when': 62,
 'time': 63,
 'into': 64,
 'these': 65,
 'only': 66,
 'see': 67,
 'may': 68,
 'than': 69,
 'i': 70,
 'world': 71,
 'b': 72,
 'd': 73,
 'would': 74,
 'no': 75,
 'however': 76,
 'between': 77,
 'about': 78,
 'over': 79,
 'states':

In [44]:
#所有单词的频数values
word_counts = np.array([count for count in vocab.values()],dtype=np.float32)

In [45]:
#所有单词的频率
word_freqs = word_counts / np.sum(word_counts)

In [46]:
#论文里乘以3/4次方
word_freqs = word_freqs **(3./4.)

In [47]:
# 重新计算所有单词的频率
word_freqs = word_freqs / np.sum(word_freqs) # 用来做negative sampling

In [48]:
# word_freqs

array([1.6231162e-02, 1.0509998e-02, 8.0359895e-03, ..., 5.0128656e-06,
       5.0128656e-06, 1.1670408e-02], dtype=float32)

In [49]:
VOCAB_SIZE = len(idx_to_word) # 词汇表单词数30000=MAX_VOCAB_SIZE

In [50]:
# VOCAB_SIZE

30000

### 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字，然后用subsampling预处理这些文字。
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

In [51]:
class WordEmbedingDataset(tud.Dataset): # 继承tud.dataset父类
    def __init__(self,text,word_to_idx,idx_to_word,word_freqs,word_counts):
        super(WordEmbedingDataset,self).__init__() # 初始化模型
        self.text_encoded = [word_to_idx.get(t,VOCAB_SIZE-1) for t in text]
        #字典 get() 函数返回指定键的值（第一个参数），如果值不在字典中返回默认值（第二个参数）。
        #取出text里每个单词word_to_idx字典里对应的索引,不在字典里返回"<unk>"的索引
        #"<unk>"的索引=29999，get括号里第二个参数应该写word_to_idx["<unk>"]，不应该写VOCAB_SIZE-1，虽然数值一样。
        
        #变成tensor类型，torch.LongTensor(self.text_encoded)
        self.text_encoded = torch.LongTensor(self.text_encoded)
        #保存数据
        self.word_to_idx = word_to_idx 
        self.idx_to_word = idx_to_word
        self.word_freqs = torch.Tensor(word_freqs)
        self.word_counts = torch.Tensor(word_counts)
        
    def __len__(self): # 数据集有多少个item
        """
        返回整个数据集（所有单词）的长度
        """
        return len(self.text_encoded)  # 所有单词的总数
    
    def __getitem__(self,idx):
        """
        返回以下数据用于训练：
        - 中心词
        - 这个单词附近的(positive)单词
        - 随机采样的K个单词作为negative sample
        """
        center_word = self.text_encoded[idx]
        #中心词索引
        #这里__getitem__函数是个迭代器，idx代表了所有的单词索引。
        
        pos_indices = list(range(idx-C,idx)) + list(range(idx+1,idx+C+1))
        #周围词的索引，比如idx=0时。pos_indices = [-3, -2, -1, 1, 2]
        
        pos_indices = [i%len(self.text_encoded) for i in pos_indices]
        #range(idx+1, idx+C+1)超出词汇总数时，需要特别处理，取余数
        
        pos_words = self.text_encoded[pos_indices]
        #周围词索引，就是希望出现的正例单词
        
        
        #负例采样单词索引，torch.multinomial作用是对self.word_freqs做K * pos_words.shape[0]次取值，输出的是self.word_freqs对应的下标。
        #取样方式采用有放回的采样，并且self.word_freqs数值越大，取样概率越大。
        #每个正确的单词采样K个，pos_words.shape[0]是正确单词数量
        #print(neg_words)
        neg_words = torch.multinomial(self.word_freqs,k*pos_word.shape[0],True)
        
        return center_word,pos_words,neg_words

创建dataset 和 dataloader

In [52]:
dataset = WordEmbedingDataset(text,word_to_idx,idx_to_word,word_freqs,word_counts)

In [53]:
dataloader = tud.DataLoader(dataset,batch_size = BATCH_SIZE,shuffle=True,num_workers=4)

### 定义PyTorch模型

In [57]:
class EmbeddingModel(nn.Module):
    def __init__(self,vocab_size,embed_size):
        """
        初始化输出 和 输出embedding
        """
        super(EmbeddingModel,self).__init__()
        self.vocab_size = vocab_size # 30000
        self.embed_size = embed_size # 100
        
        initrange = 0.5 / self.embed_size
        self.out_embed = nn.Embedding(self.vocab_size,self.embed_size,sparse=False)
        #模型输出nn.Embedding(30000, 100)
        self.out_embed.weight.data.uniform_(-initrange,initrange)
        #权重初始化的一种方法
        
        self.in_embed = nn.Embedding(self.vocab_size,self.embed_size,sparse=False)
        self.in_embed.weight.data.uniform_(-initrange,initrange)
        
    def forward(self,input_labels,pos_labels,neg_labels):
        """
        input_labels: 中心词, [batch_size]
        pos_labels: 中心词周围 context window 出现过的单词 [batch_size * (window_size * 2)]
        neg_labelss: 中心词周围没有出现过的单词，从 negative sampling 得到 [batch_size, (window_size * 2 * K)]
        return loss,[batch_size]
        """
        batch_size = input_labels.size(0)
        #input_labels是输入的标签，tud.DataLoader()返回的。相已经被分成batch了
        
        input_embedding = self.in_embed(input_labels)
        # B * embed_size
        #这里进行了运算：（128,30000）*（30000,100）= 128(B) * 100 (embed_size)
        
        pos_embedding = self.out_embed(pos_labels)  # B * (2*C) * embed_size
        #同上，增加了维度(2*C)，表示一个batch有B组周围词单词，一组周围词有(2*C)个单词，每个单词有embed_size个维度。
        
        neg_embedding = self.out_embed(neg_labels) #B * (2*C * K) * embed_size
        #同上，增加了维度(2*C*K)
        
        #torch.bmm()为batch间的矩阵相乘（b,n.m)*(b,m,p)=(b,n,p)
        log_pos = torch.bmm(pos_embedding,input_embedding.unsqueeze(2)).squeeze() # B*(2*c)
        log_neg = torch.bmm(neg_embedding,-input_embedding.unsqueeze(2)).squeeze() # B*(*C*K)
        #unsqueeze(2)指定位置升维，.squeeze()压缩维度
        
        log_pos = F.logsigmoid(log_pos).sum(1)
        log_neg = F.logsigmoid(log_neg).sum(1) # batch_size
        loss = log_pos + log_neg
        
        return -loss

In [58]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [62]:
model = EmbeddingModel(VOCAB_SIZE,EMBEDDING_SIZE)
model= model.to(device)
optimizer = torch.optim.SGD(model.parameters(),lr = LEARNING_RATE)

模型评估

In [60]:
def evaluate(filename,embedding_weights):
    if filename.endswith('.csv'):
        data = pd.read_csv(filename,sep=',')
    else:
        data = pd.read_csv(filename,sep='\t')
    human_similarity = []
    model_similarity = []
    
    for i in data.iloc[:,0:2].index:
        word1,word2 = data.iloc[i,0],data.iloc[i,1]
        if word1 not in word_to_idx or word2 not in word_to_idx:
            continue
        else:
            word1_idx,word2_idx = word_to_idx[word1],word_to_idx[word2]
            word1_embed,word2_embed = embedding_weights[[word1_idx]],embedding_weights[[word2_idx]]
            model_similarity.append(float(sklearn.metrics.pairwise.cosine_similarity(word1_embed,word2_embed)))
            human_similarity.append(float(data.iloc[i,2]))
            
    return scipy.stats.spearmanr(human_similarity,model_similarity)


In [61]:
def find_nearest(word):
    index = word_to_idx[word]
    embedding = embedding_weights[index]
    cos_dis = np.array([scipy.spatial.distance.cosine(e,embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]]

训练模型：
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，以及在验证数据集上做模型的评估

In [ ]:
# for i,(input_labels,pos_labels,neg_labels) in enumerate(dataloader):
#     if i > 5:
#         break
for e in range(NUM_EPOCHS):
    for i,(input_labels,pos_labels,neg_labels) in enumerate(dataloader):
        input_labels = torch.LongTensor(input_labels).to(device)
        pos_labels = torch.LongTensor(pos_labels).to(device)
        neg_labels = torch.LongTensor(neg_labels).to(device)
        
        loss = model(input_labels,pos_labels,neg_labels).mean()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%100 == 0:
            with open(LOG_FILE,'a') as fout:
                fout.write('epoch:{},iter:{},loss:{}\n'.format(e,i,loss.item()))
                print('epoch:{},iter:{},loss:{}\n'.format(e,i,loss.item()))
                
        if i%2000 == 0:
            embedding_weights = model.input_embeddings()
            sim_simlex = evaluate('./simlex-999.txt',embedding_weights)
            sim_men = evaluate('./men.txt',embedding_weights)
            sim_353 = evaluate('./wordsim353.txt')
            with open(LOG_FILE, "a") as fout:
                print("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                fout.write("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
        
    embedding_weights = model.input_embeddings()
    np.save("embedding-{}".format(EMBEDDING_SIZE), embedding_weights)
    torch.save(model.state_dict(), "embedding-{}.th".format(EMBEDDING_SIZE))